![Banner](images/banner.png)

# Transaction Mananagement and Sessionless Transactions

<hr>

Setup for this notebook:

In [ ]:
import os
import oracledb

un = os.environ.get("PYO_SAMPLES_MAIN_USER", "pythondemo")
pw = os.environ.get("PYO_SAMPLES_MAIN_PASSWORD", "welcome")
cs = os.environ.get("PYO_SAMPLES_CONNECT_STRING", "localhost/orclpdb")

connection = oracledb.connect(user=un, password=pw, dsn=cs)

cursor = connection.cursor()
try:
    cursor.execute("drop table mytab")
except:
    pass
cursor.execute("create table mytab (id number, data varchar2(1000))")

## Transaction Management

Documentation reference link: [Managing Transations](https://python-oracledb.readthedocs.io/en/latest/user_guide/txn_management.html).

A database transaction is a grouping of SQL statements that make a logical data change to the database. When statements like `Cursor.execute()` or `Cursor.executemany()` execute SQL statements like INSERT or UPDATE, a transaction is started or continued. 

In [ ]:
connection1 = oracledb.connect(user=un, password=pw, dsn=cs)
cursor1 = connection1.cursor()
cursor1.execute("insert into mytab (id, data) values (1, 'Widget')")

By default, python-oracledb does not commit this transaction to the database. 

In [ ]:
connection2 = oracledb.connect(user=un, password=pw, dsn=cs)
cursor2 = connection2.cursor()

# No rows will be displayed
rows = cursor2.execute("select * from mytab")
for r in rows:
    print(r)

You can explictly commit or roll it back using the methods `Connection.commit()` and `Connection.rollback()`.  

To commit the row just inserted:

In [ ]:
connection1.commit()

The committed data is now visible in the second connection:

In [ ]:
for r in cursor2.execute("select * from mytab"):
    print(r)

In [ ]:
# Delete the data so the example can be re-run

cursor1.execute("truncate table mytab")
connection1.commit()

## Autocommitting

Documentation reference link: [Autocommitting](https://python-oracledb.readthedocs.io/en/latest/user_guide/txn_management.html#autocommitting).

Autocommitting is a way to improve application scalability by removing the need to perform an explicit `connection.commit()` call.

The recommendation is to use it on the last DML statement of a transaction.

In [ ]:
connection1 = oracledb.connect(user=un, password=pw, dsn=cs)
cursor1 = connection1.cursor()

connection1.autocommit = False  # make sure the value is off
cursor1.execute("insert into mytab (id, data) values (1, 'Widget')")

# Insert a second row and commit the transaction
connection1.autocommit = True
cursor1.execute("insert into mytab (id, data) values (2, 'Thingummy')")

Other sessions will see the committed data:

In [ ]:
connection2 = oracledb.connect(user=un, password=pw, dsn=cs)
cursor2 = connection2.cursor()

for r in cursor2.execute("select * from mytab"):
    print(r)

You should avoid the over-use of autocommit because it can break relational consistency. Also unnecessarily committing will add additional load the the database.

In [ ]:
# Delete the data so the example can be re-run

cursor1.execute("truncate table mytab")
connection1.commit()

## Sessionless Transactions

Documentation reference link: [Sessionless Transactions](https://python-oracledb.readthedocs.io/en/latest/user_guide/txn_management.html#sessionless-transactions).

A Sessionless Transaction is a transaction that can be suspended and resumed during its lifecycle. A transaction is no longer tied to a specific connection. This enables connections to be released for use by other users while a transaction remains open to be resumed later. One use case is with web services, where a user's workflow involves calling individual services for specific tasks, and each service uses different tables for their part of the workflow. Each service can start, suspend, and resume a transaction as needed to complete the workflow in a single transaction, allowing relational consistency to be ensured. Another scenario is when a customer has some 'think time' between making purchases from an online store. During that intermediate time the customer is browsing the site, the connection can be released for other customers to use. This reduces the overall system resources required.

With Sessionless Transactions, you do not need to use a transaction manager since Oracle Database manages coordination of transactions.

In [ ]:
# Start a transaction
connection1 = oracledb.connect(user=un, password=pw, dsn=cs)
txn_id = connection1.begin_sessionless_transaction()
cursor1 = connection1.cursor()
cursor1.execute("insert into mytab (id, data) values (1, 'Blue Dress')")
connection1.suspend_sessionless_transaction()

# The user might have some 'think time' before doing more database work on the transaction
# ...

# In the same or another process, a second connection can resume and complete the transaction
connection2 = oracledb.connect(user=un, password=pw, dsn=cs)
connection2.resume_sessionless_transaction(txn_id)
cursor2 = connection2.cursor()
cursor2.execute("insert into mytab (id, data) values (2, 'Green Shirt')")
connection2.commit()


Both records have been committed to the database:

In [ ]:
connection3 = oracledb.connect(user=un, password=pw, dsn=cs)
cursor3 = connection3.cursor()

for r in cursor3.execute("select * from mytab"):
    print(r)

In [ ]:
# Delete the data so the example can be re-run

cursor1.execute("truncate table mytab")
connection1.commit()